## CASSDA - Data Download

Contains code to download the following data
- Wildlife crossings from OSM
- Streets from OSM
- High-Resolution satellite images from discomap https://discomap.eea.europa.eu/wiki/ (Cloudless 2m/px satellite data)

In [1]:
import sys
import os

sys.path.append("../src")

import requests as r
from shapely.geometry import LineString, MultiPolygon, Point, Polygon
import geopandas as gpd

from osm.queries import get_street_query, get_wildlife_crossing_query

#### Global Config

In [2]:
countries_to_process = ["CH"]
image_output_dir = "../data/satellite_images/"
geodata_output_dir = "../data/geodata/"

In [3]:
# these typically do not need to be changed
OVERPASS_API_URL = "https://overpass-api.de/api/interpreter"
WMS_URL = "https://image.discomap.eea.europa.eu/arcgis/services/GioLand/VHR_2021_LAEA/ImageServer/WMSServer/?request=GetCapabilities&service=WMS"
CRS = "EPSG:3035"  # ETRS89 / ETRS-LAEA
PIXEL_RESOLUTION_M = 2  # 2 meters per pixel

### Wildlife Crossing

#### Export Polygons from OSM

In [ ]:
# config for wildlife crossing query
bridges_only = True  # whether to only consider bridges or tunnels and other as well

Gets the queries according to defined parameters, sends it co Overpass-API and 
on success stores the result in <geodata_output_dir>

In [8]:
bridges_string = "_bridges_only" if bridges_only else ""

for country in countries_to_process:
    print(f"Processing country: {country}")

    # Download wildlife crossings
    crossing_query = get_wildlife_crossing_query(
        country, timeout=240, bridges_only=bridges_only
    )
    response = r.get(OVERPASS_API_URL, params={"data": crossing_query})
    response.raise_for_status()
    data = response.json()

    features = []
    for el in data["elements"]:
        tags = el.get("tags", {})

        # Node → Point
        if el["type"] == "node":
            geometry = Point(el["lon"], el["lat"])
            features.append({"id": el["id"], "geometry": geometry, **tags})

        # Way → LineString or Polygon
        elif el["type"] == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 3 and coords[0] == coords[-1]:
                geometry = Polygon(coords)  # Closed → Polygon
            else:
                geometry = LineString(coords)  # Open → LineString
            features.append({"id": el["id"], "geometry": geometry, **tags})

        # Relation → MultiPolygon (if members have geometry)
        elif el["type"] == "relation" and "members" in el:
            polygons = []
            for member in el["members"]:
                if member["type"] == "way" and "geometry" in member:
                    coords = [(pt["lon"], pt["lat"]) for pt in member["geometry"]]
                    if len(coords) >= 3:
                        polygons.append(Polygon(coords))
            if polygons:
                geometry = MultiPolygon(polygons) if len(polygons) > 1 else polygons[0]
                features.append({"id": el["id"], "geometry": geometry, **tags})

    # --- Create GeoDataFrame ---
    if features:
        print(f"Found {len(features)} wildlife crossings.")
        gdf = gpd.GeoDataFrame(features, geometry="geometry", crs="EPSG:4326")
    else:
        print("No wildlife crossings found.")

    # --- Reproject to EPSG:3035 (ETRS89 / LAEA Europe) ---
    if not gdf.empty:
        gdf = gdf.to_crs(epsg=3035)
        print("Reprojected to EPSG:3035.")

    os.makedirs(geodata_output_dir, exist_ok=True)

    # Save the GeoDataFrame to a file
    output_file = os.path.join(
        geodata_output_dir,
        f"{country.lower()}_wildlife_crossings_osm{bridges_string}.geojson",
    )
    gdf.to_file(output_file, driver="GeoJSON")
    print(f"Saved GeoDataFrame to {output_file}")

Processing country: CH
Found 18 wildlife crossings.
Reprojected to EPSG:3035.
Saved GeoDataFrame to ../data/geodata/ch_wildlife_crossings_osm_bridges_only.geojson


#### Download Images from WMS

In [ ]:
object_source = ""
geodata_output_dir = "../data/geodata/"